## 1. Dividindo os dados entre treino e teste

In [ ]:
import pandas as pd

In [2]:
treino = pd.read_csv('https://raw.githubusercontent.com/cassiasamp/calculadora-de-imoveis-jun-20/master/analise_e_preprocessamento/treino_preprocessado.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/cassiasamp/calculadora-de-imoveis-jun-20/master/analise_e_preprocessamento/teste_preprocessado.csv')

In [3]:
treino.head()

,zona_leste,zona_norte,zona_oeste,zona_sul,quartos,area,preco
0,0,1,0,0,0.693147,3.713572,6.621406
1,1,0,0,0,1.098612,5.198497,7.601402
2,0,0,0,1,1.386294,6.216606,8.699681
3,0,1,0,0,1.098612,4.262680,7.090910
4,1,0,0,0,1.098612,4.189655,7.313887


In [4]:
teste.head()

,zona_leste,zona_norte,zona_oeste,zona_sul,quartos,area,preco
0,0,1,0,0,1.098612,6.111467,9.903538
1,1,0,0,0,1.098612,5.525453,8.853808
2,0,0,1,0,1.609438,5.802118,9.852247
3,0,0,0,1,1.386294,5.888878,9.350189
4,0,1,0,0,1.386294,4.394449,7.003974


In [5]:
X_treino = treino.drop('preco', axis=1)

In [6]:
X_treino.head()

,zona_leste,zona_norte,zona_oeste,zona_sul,quartos,area
0,0,1,0,0,0.693147,3.713572
1,1,0,0,0,1.098612,5.198497
2,0,0,0,1,1.386294,6.216606
3,0,1,0,0,1.098612,4.262680
4,1,0,0,0,1.098612,4.189655


In [7]:
y_treino = treino['preco']

In [8]:
y_treino.head()

0    6.621406
1    7.601402
2    8.699681
3    7.090910
4    7.313887
Name: preco, dtype: float64

In [9]:
X_teste = teste.drop('preco', axis=1)
y_teste = teste['preco']

## 2. Criando um modelo de regressão linear

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
reg = LinearRegression()

In [12]:
reg.fit(X_treino, y_treino)

LinearRegression()

In [24]:
import numpy as np

# reg.predict(X_teste)
n_quartos = 3
area = 100

log1_pred = reg.predict([[0, 1, 0, 0, np.log1p(n_quartos), np.log1p(area)]])

Estamos lidando com um modelo que é log-log, mais especificamente, log + 1 de entrada e exponencial - 1 de saída.

In [25]:
np.expm1(log1_pred)

array([1908.67337774])

In [27]:
log1_predicoes = reg.predict(X_teste)

In [30]:
predicoes = np.expm1(log1_predicoes)

In [31]:
X_teste.head()

,zona_leste,zona_norte,zona_oeste,zona_sul,quartos,area
0,0,1,0,0,1.098612,6.111467
1,1,0,0,0,1.098612,5.525453
2,0,0,1,0,1.609438,5.802118
3,0,0,0,1,1.386294,5.888878
4,0,1,0,0,1.386294,4.394449


In [32]:
y_teste.head()

0    9.903538
1    8.853808
2    9.852247
3    9.350189
4    7.003974
Name: preco, dtype: float64

In [34]:
np.log1p(predicoes[0])

8.5703125

In [35]:
9.9 - 8.5

1.4000000000000004

In [39]:
reg.score(X_teste, y_teste) # r quadrado da regressão linear

0.7529715525631803

## 3. Criando um modelo de base (baseline, benchmark)

In [40]:
# modelo que preveja a média
from sklearn.dummy import DummyRegressor

In [41]:
regressor_boboca = DummyRegressor(strategy='mean')

In [42]:
regressor_boboca.fit(X_treino, y_treino)

DummyRegressor()

In [43]:
regressor_boboca.score(X_teste, y_teste) # quando a gente faz o score, usamos x e y de teste

-0.007175323673028533

In [45]:
pred_boboca = regressor_boboca.predict(X_teste) # quando a gente faz a predição, só passamos o x de teste

Quando a gente usa o R² de dentro do pacote de métricas do sklearn (ou qualquer outra métrica do pacote metrics) passamos o y_true (y_teste) e os valores preditos (y_pred).

O .score é o R² do pacote de métricas.

In [46]:
from sklearn.metrics import r2_score

r2_score(y_teste, pred_boboca)

-0.007175323673028533

In [57]:
from sklearn.ensemble import RandomForestRegressor

rfn = RandomForestRegressor(random_state=100)
rfn.fit(X_treino, y_treino)
rfn.score(X_teste, y_teste)

0.7434486790195538

In [59]:
# 42, 1, 100, 0 .... # vemos na próxima aula

## 4. Salvando o modelo (exportando o modelo)

In [60]:
import pickle

In [74]:
pickle.dump(rfn, open('modelo_somos_nozes.pkl', 'wb')) # wb é de escrever o binário ou write binary 

In [75]:
rfr = pickle.load(open('modelo_somos_nozes.pkl', 'rb')) # rb é de ler o binário ou read binary

In [70]:
primeiro_el_teste = X_teste.values[0]

In [71]:
rfr.predict([primeiro_el_teste])

array([8.93519816])

In [72]:
el_teste_verdadeiro = y_teste.values[0]

In [73]:
el_teste_verdadeiro

9.90353755128617